In [ ]:
'at_fault': #OK
'case_id': #OK
'cellphone_use': # Problem: une chiée de '1', '2', '3' => deviner à quoi ça correspond ?? 
'financial_responsibility': #OK
'hazardous_materials': #OK,
'id': #OK
'movement_preceding_collision': #OK
'other_associate_factor_1': #OK,
'other_associate_factor_2': #OK,
'party_age': pd.Int16Dtype(), # Problem: interpretation age 999 => pregnant
'party_drug_physical': str, # Problem: 585062 rows with G corresponding to nothing => replace by None ? 
'party_number': #OK
'party_safety_equipment_1': #OK,
'party_safety_equipment_2': #OK,
'party_sex': #OK
'party_sobriety': #OK
'party_type': #OK
'school_bus_related': #OK
'statewide_vehicle_type': #OK
'vehicle_make': # Koi kil faut check ?
'vehicle_year':#OK,

In [1]:
import pandas as pd
import numpy as np

In [58]:
dtype_parties={
    'at_fault': bool, # bool or floats
    'case_id': np.uint64, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'cellphone_use': str, 
    'financial_responsibility': str,
    'hazardous_materials': str,
    'id': int,
    'movement_preceding_collision': str,
    'other_associate_factor_1': str,
    'other_associate_factor_2': str,
    'party_age': pd.Int16Dtype(),
    'party_drug_physical': str,
    'party_number': int,
    'party_safety_equipment_1': str,
    'party_safety_equipment_2': str,
    'party_sex': str,
    'party_sobriety': str,
    'party_type': str,
    'school_bus_related': str,
    'statewide_vehicle_type': str,
    'vehicle_make': str,
    'vehicle_year': pd.Int16Dtype(),
}#check what type to give for integers
#int does not allow nulls
#np.uint64 allows up to 64 bytes (necessary for case id)
#pd.Int16/32/64Dtype() allows nulls

In [241]:
nrows = 5

###### Delete nrows ######

sep=',' by default 

header='0' by default (first line is column names)

should we use index_col=0 for collisions data

usecols to make sure we're reading correct columns

add dtype

use converters : Dict of functions for converting values in certain columns. Keys can either be integers or column labels.

??na_values / keep_default_na / na_filter

check with sql if need to infer_datetime_format/keep_date_col/date_parser/dayfirst/cache_dates

comment is it useful?

error_bad_lines/warn_bad_lines

In [242]:
#read last line
# f1 = open("CSV-2018/parties2018.csv", "r")
# last_line = f1.readlines()[-1]
# print(last_line)
# f1.close()

0,9870011231152508671,,Y,,16432243,"making left turn",A,,64,,2,G,,male,A,driver,,"passenger car",CHRYSLER,1987



7286606

In [59]:
parties_data = pd.read_csv("CSV-2018/parties2018.csv",nrows=7286606,dtype=dtype_parties,usecols=dtype_parties.keys()) 

In [7]:
parties_data

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year
0,True,1,C,Y,NaN,8,other,L,NaN,84,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,2000
1,False,1,D,O,NaN,9,parked,N,NaN,<NA>,...,2,NaN,NaN,NaN,H,parked vehicle,NaN,passenger car,BUICK,1992
2,False,2,NaN,O,NaN,10,proceeding straight,O,NaN,<NA>,...,1,NaN,NaN,NaN,H,other,NaN,pickup or panel truck,TOYOTA,NaN
3,False,3,D,Y,NaN,11,proceeding straight,N,NaN,20,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,1995
4,True,3,D,NaN,NaN,12,proceeding straight,N,NaN,<NA>,...,2,NaN,NaN,NaN,G,driver,NaN,pickup or panel truck,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286601,True,9870011226102009803,D,Y,NaN,16432239,making left turn,N,NaN,66,...,1,G,NaN,male,A,driver,NaN,passenger car,CADILLAC,1994
7286602,False,9870011226102009803,D,Y,NaN,16432240,proceeding straight,N,NaN,20,...,2,G,NaN,female,A,driver,NaN,passenger car,CHRYSLER,1991
7286603,True,9870011228210011458,D,N,NaN,16432241,ran off road,N,NaN,<NA>,...,1,B,NaN,NaN,G,driver,NaN,passenger car,CHEVROLET,NaN
7286604,True,9870011231152508671,NaN,Y,NaN,16432242,passing other vehicle,A,F,19,...,1,G,NaN,female,A,driver,NaN,passenger car,CADILLAC,1991


In [21]:
clean_data = parties_data.copy()

In [53]:
translation_movement_preceding_collision ={
    'stopped': "A",
    'proceeding straight': "B",
    'ran off road': "C",
    'making right turn': "D",
    'making left turn': "E",
    'making u-turn': "F",
    'backing': "G",
    'slowing/stopping': "H",
    'passing other vehicle': "I",
    'changing lanes': "J",
    'parking maneuver': "K",
    'entering traffic': "L",
    'other unsafe turning': "M",
    'crossed into opposing lane': "N",
    'parked': "O",
    'merging': "P",
    'traveling wrong way': "Q",
    'other': "R",
}

translation_party_sex ={
    'female': "F",
    'male': "M",
}

translation_party_type ={
    'driver': 1,
    'pedestrian': 2,
    'parked vehicle': 3,
    'bicyclist': 4, 
    'other': 5,
}

translation_statewide_vehicle_type ={
    'passenger car': "A",
    'passenger car with trailer':"B",
    'motorcycle or scooter': "C",
    'pickup or panel truck': "D",
    'pickup or panel truck with trailer': "E",
    'truck or truck tractor': "F",
    'truck or truck tractor with trailer': "G",
    'schoolbus': "H",
    'other bus': "I",
    'emergency vehicle': "J",
    'highway construction equipment': "K",
    'bicycle': "L",
    'other vehicle':"M",
    'pedestrian': "N",
    'moped':"O", 
}

In [54]:
clean_data["movement_preceding_collision"] = parties_data["movement_preceding_collision"].map(lambda s: translation_movement_preceding_collision.get(s, None))
clean_data["party_sex"] = parties_data["party_sex"].map(lambda s: translation_party_sex.get(s, None))
clean_data["party_type"] = parties_data["party_type"].map(lambda s: translation_party_type.get(s, None)).astype(pd.Int16Dtype())
clean_data["statewide_vehicle_type"] = parties_data["statewide_vehicle_type"].map(lambda s: translation_statewide_vehicle_type.get(s, None))


In [57]:
set(parties_data["vehicle_year"])

{'1903',
 '1904',
 '1905',
 '1906',
 '1907',
 '1908',
 '1909',
 '1910',
 '1911',
 '1912',
 '1914',
 '1915',
 '1916',
 '1917',
 '1918',
 '1919',
 '1920',
 '1921',
 '1922',
 '1923',
 '1924',
 '1925',
 '1926',
 '1927',
 '1928',
 '1929',
 '1930',
 '1931',
 '1932',
 '1933',
 '1934',
 '1935',
 '1936',
 '1937',
 '1938',
 '1939',
 '1940',
 '1941',
 '1942',
 '1943',
 '1944',
 '1945',
 '1946',
 '1947',
 '1948',
 '1949',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2014',
 '2015',
 

In [55]:
set(clean_data["statewide_vehicle_type"])

{'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 None,
 'O'}

In [60]:
#parties_data[parties_data["party_drug_physical"]== "G"]

parties_data[parties_data["case_id"].isnull()]

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year


In [16]:
parties_data[parties_data["cellphone_use"]=='3']

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year
747643,True,407134,3,NaN,NaN,747654,backing,N,NaN,<NA>,...,1,NaN,NaN,NaN,A,driver,NaN,NaN,NaN,NaN
747644,False,407134,3,O,NaN,747655,parked,N,NaN,<NA>,...,2,NaN,NaN,NaN,A,parked vehicle,NaN,NaN,HONDA,1993
1581256,True,875345,3,Y,NaN,1581271,other,L,NaN,39,...,1,P,G,female,A,driver,NaN,passenger car,FORD,2001
1604650,True,888977,3,Y,NaN,1604665,ran off road,F,NaN,19,...,1,P,G,male,B,driver,NaN,pickup or panel truck,FORD,1984
1624976,True,900641,3,NaN,NaN,1624991,changing lanes,N,NaN,<NA>,...,2,B,NaN,male,G,driver,NaN,NaN,CHEVROLET,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859779,True,90816783,3,N,NaN,15998489,proceeding straight,N,NaN,17,...,1,L,G,male,A,driver,NaN,passenger car,FORD,1995
6859780,False,90816783,3,Y,NaN,15998490,proceeding straight,N,NaN,57,...,2,L,G,female,A,driver,NaN,passenger car,NISS,2012
6859781,True,90818829,3,Y,NaN,16002518,making left turn,N,NaN,80,...,1,L,G,male,A,driver,NaN,passenger car,DODG,2000
7181720,True,965874,3,Y,NaN,6,proceeding straight,N,NaN,26,...,1,M,G,female,A,driver,NaN,passenger car,HYUNDAI,1999


In [248]:
print(parties_data[7286605:]['case_id'])
print(parties_data[7286605:]['id'])
print(parties_data[7286605:]['party_age'])
print(parties_data[7286605:]['party_number'])

7286605    9870011231152508671
Name: case_id, dtype: uint64
7286605    16432243
Name: id, dtype: int64
7286605    64
Name: party_age, dtype: Int16
7286605    2
Name: party_number, dtype: int64


                                ------------------------------------------

In [250]:
victims_data = pd.read_csv("CSV-2018/victims2018.csv", nrows=nrows) 

In [251]:
f1 = open("CSV-2018/parties2018.csv", "r")
last_line = f1.readlines()[-1]
print(last_line)
f1.close()

0,9870011231152508671,,Y,,16432243,"making left turn",A,,64,,2,G,,male,A,driver,,"passenger car",CHRYSLER,1987



In [208]:
collisions_data = pd.read_csv("CSV-2018/collisions2018.csv", nrows=nrows) 

In [41]:
# print(victims_data)

In [42]:
# print(collisions_data)

In [43]:
case_id = 5
parties_data_case = parties_data[parties_data["case_id"] == case_id]
victims_data_case = victims_data[victims_data["case_id"] == case_id]
collisions_data_case = collisions_data[collisions_data["case_id"] == case_id]

In [44]:
parties_data_case

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year


In [42]:
victims_data_case

,case_id,id,party_number,victim_age,victim_degree_of_injury,victim_ejected,victim_role,victim_safety_equipment_1,victim_safety_equipment_2,victim_seating_position,victim_sex
1,5.0,4,1,44.0,severe injury,0.0,1,G,NaN,1.0,male


In [43]:
collisions_data_case

,case_id,collision_date,collision_severity,collision_time,county_city_location,hit_and_run,jurisdiction,lighting,location_type,officer_id,...,primary_collision_factor,process_date,ramp_intersection,road_condition_1,road_condition_2,road_surface,tow_away,type_of_collision,weather_1,weather_2
4,5.0,2002-02-14,severe injury,11:35:00,1500,not hit and run,9420.0,daylight,NaN,13851,...,vehicle code violation,2003-07-28,NaN,normal,NaN,dry,1.0,overturned,clear,NaN


In [86]:
options = ["B", "C", "D", None]
parties_data_test = parties_data[~(parties_data["cellphone_use"].isin(options) | parties_data["cellphone_use"].isnull().values.any())]



In [87]:
parties_data_test

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year


In [68]:
len(parties_data)

7286606